#Generative AI in Musicology: Structured Analysis and Creative Synthesis

##Song used: All I want for Christmas is You by Mariah Carrey

**boot up**

*   inilizes functions time, userdata, os, json, and time
*   checks for gemni key to make sure that it works
*   checks for text file with the song lyrics
*   initlizes dictionary called song data which has song name, artist, and text file of lryics

this uses try and except blocks so it will let you know if something doesn't inialize properly without ending the program. the dictionary will print no matter what so it is good to make sure that everything was trasfered properly





In [ ]:
import google.generativeai as genai
from google.colab import userdata
import os # For robust file path handling
import json # For structured data handling
import time # For the exponential backoff delay

# --- 1.1 API Configuration ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configured successfully.")

    # Initialize the Gemini model
    gemini_model = genai.GenerativeModel('gemini-flash-latest')
    print("Model 'gemini-flash-latest' initialized.")

except Exception as e:
    print(f"Error during API configuration: {e}")

# --- 1.2 Source Data I/O ---
LYRICS_FILENAME = "song_lyrics.txt"
song_lyrics = ""

try:
    # Robust File I/O using a try...except block to handle FileNotFoundError
    with open(LYRICS_FILENAME, 'r', encoding='utf-8') as f:
        song_lyrics = f.read()
    print(f"\nSuccessfully read lyrics from: {LYRICS_FILENAME}")
    print(f"File content length: {len(song_lyrics)} characters.")

except FileNotFoundError:
    print(f"\nError: The file '{LYRICS_FILENAME}' was not found.")
    print("Please make sure the file is in the correct directory.")
    # Set lyrics to a known error state to prevent downstream failures
    song_lyrics = "LYRICS_FILE_NOT_FOUND_ERROR"
except Exception as e:
    print(f"\nAn unexpected error occurred during file reading: {e}")
    song_lyrics = "UNEXPECTED_IO_ERROR"

# --- 1.3 Data Initialization ---
# Create the core Python song dictionary to store initial data
song_data = {
    "title": "All I want for Christmas Is You",
    "artist": "Mariah Carey",
    "lyrics": song_lyrics }

print("\n--- Initial Song Dictionary (Stage 1 Output) ---")
print(song_data)
print("-------------------------------------------------")

Gemini API configured successfully.
Model 'gemini-flash-latest' initialized.

Successfully read lyrics from: song_lyrics.txt
File content length: 1568 characters.

--- Initial Song Dictionary (Stage 1 Output) ---
{'title': 'All I want for Christmas Is You', 'artist': 'Mariah Carey', 'lyrics': "I don't want a lot for Christmas\n\nThere is just one thing I need\n\nI don't care about the presents\n\nUnderneath the Christmas tree\n\n\n\nI just want you for my own\n\nMore than you could ever know\n\nMake my wish come true\n\nAll I want for Christmas is you, yeah\n\n\n\nI don't want a lot for Christmas\n\nThere is just one thing I need\n\nAnd I don't care about the presents\n\nUnderneath the Christmas tree\n\n\n\nI don't need to hang my stocking\n\nThere upon the fireplace\n\nSanta Claus won't make me happy\n\nWith a toy on Christmas Day\n\n\n\nI just want you for my own\n\nMore than you could ever know\n\nMake my wish come true\n\nAll I want for Christmas is you\n\nYou, baby\n\n\n\nOh, I wo

**call gemini function - stage 2**

this is the function that prompts the acutal Ai itself using the generate_content function. it begins by defining a for loop with the amount of retires with a defualt of 3 and another intake for the prompt itself. it runs everthing through a for loop so that if one prompt fails then it still tries to answer the querry

In [ ]:
def call_gemini_with_retry(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = gemini_model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Error on attempt {attempt+1}: {e}")

            # Only retry if not on the last attempt
            if attempt < max_retries - 1:
                pass
            else:
                return "could not generate content."

**Prompt - stage 2**

this prompt adds a title, songwirter, and album title. it is written in the format so that it fills in a number of dictonary terms and then adds them to the song data function. it is important to have this written in JSON as it is meant to be passed through files

In [ ]:
metadata_prompt = f"""
You are an expert in music metadata analysis.

Given the following lyrics, extract ONLY the following fields in STRICT JSON format:

{{
  "title": "...",
  "album title": "..",
  "songwriters": "..."
  "release_date": "YYYY-MM-DD or 'unknown'"
}}

Rules:
- Respond with ONLY valid JSON.
- If any field is uncertain, return "unknown".
- Do not include commentary.
- Do not include ANY text outside the JSON object.
- This is going into a dictionary so make sure to use double quotes.

Lyrics:
\"\"\"{song_data['lyrics']}\"\"\"
"""


**printer - stage 2**

this brings the result from the prompt which is a JSON file and then brings it throught the dictionary and prints the result to make sure everything transfered properly

In [ ]:
raw_metadata_json = call_gemini_with_retry(metadata_prompt)
try:
    metadata = json.loads(raw_metadata_json)
    print("Successfully parsed Stage 2 metadata JSON.")
    song_data.update(metadata)

except json.JSONDecodeError as e:
    print("JSON decoding error in Stage 2:", e)
for i in song_data:
  print(i,":", song_data[i])


Successfully parsed Stage 2 metadata JSON.
title : All I Want for Christmas Is You
artist : Mariah Carey
lyrics : I don't want a lot for Christmas

There is just one thing I need

I don't care about the presents

Underneath the Christmas tree



I just want you for my own

More than you could ever know

Make my wish come true

All I want for Christmas is you, yeah



I don't want a lot for Christmas

There is just one thing I need

And I don't care about the presents

Underneath the Christmas tree



I don't need to hang my stocking

There upon the fireplace

Santa Claus won't make me happy

With a toy on Christmas Day



I just want you for my own

More than you could ever know

Make my wish come true

All I want for Christmas is you

You, baby



Oh, I won't ask for much this Christmas

I won't even wish for snow

And I'm just gonna keep on waiting

Underneath the mistletoe



I won't make a list and send it

To the North Pole for Saint Nick

I won't even stay awake to

Hear those ma

**prompt - stage 3**

this prompt functions similarly to the last with adding more keys to the dictionary. there are more rules for the ai to follow like rules for the summary and the requirment not to consider stopwords from nltk in the top ten words function. this code block also imports the files

In [ ]:
from nltk.corpus import stopwords
metadata_prompt2 = f"""
You are an expert in music metadata analysis.

Given the following lyrics, extract ONLY the following fields in STRICT JSON format:

{{
  "Genre": ["...","...","..."],
  "Explicit": "Yes or No",
  "Topwords": ["...","...","...","...","...","...","...","...","...","..."]
  "release_date": "YYYY-MM-DD or 'unknown'"
  "summary": "..."
  "sentiments": ['Hopeful', 'Determined', 'Affectionate', 'Restless', 'Adventurous']
  "themes_topics": ['New Beginnings', 'Starting Over', 'Taking Risks', 'Travel and
Escapism', 'Commitment in Partnership', 'Perseverance']


}}

Rules:
- Respond with ONLY valid JSON.
- If any field is uncertain, return "unknown".
- Do not include commentary.
- Do not include ANY text outside the JSON object.
- This is going into a dictionary so make sure to use double quotes.
-summary should have supporting evidence from the lyrics.
-anything from nltk.corpus import stopwords should not be included in your top ten words

Lyrics:
\"\"\"{song_data['lyrics']}\"\"\"
"""

In [ ]:
raw_metadata_json = call_gemini_with_retry(metadata_prompt2)
try:
    metadata = json.loads(raw_metadata_json)
    print("Successfully parsed Stage 2 metadata JSON.")
    song_data.update(metadata)

except json.JSONDecodeError as e:
    print("JSON decoding error in Stage 2:", e)
for i in song_data:
  print(i,":", song_data[i])

Successfully parsed Stage 2 metadata JSON.
title : All I Want for Christmas Is You
artist : Mariah Carey
lyrics : I don't want a lot for Christmas

There is just one thing I need

I don't care about the presents

Underneath the Christmas tree



I just want you for my own

More than you could ever know

Make my wish come true

All I want for Christmas is you, yeah



I don't want a lot for Christmas

There is just one thing I need

And I don't care about the presents

Underneath the Christmas tree



I don't need to hang my stocking

There upon the fireplace

Santa Claus won't make me happy

With a toy on Christmas Day



I just want you for my own

More than you could ever know

Make my wish come true

All I want for Christmas is you

You, baby



Oh, I won't ask for much this Christmas

I won't even wish for snow

And I'm just gonna keep on waiting

Underneath the mistletoe



I won't make a list and send it

To the North Pole for Saint Nick

I won't even stay awake to

Hear those ma

**prompt run:**

runs prompt in the same way as the previous

In [ ]:
recommendation_prompt = f"""
You are an expert music recommendation system.

I need 7 song recommendations based on the following metadata. Please note that the original lyrics were not available, so base your recommendations solely on the provided metadata fields. If any field is 'unknown' or indicates an error, please ignore it and focus on the available information.

Original Song Information:
  - Title: {song_data['title']}
  - Artist: {song_data['artist']}
  - Genre: {song_data['Genre']}
  - Sentiments: {song_data['sentiments']}
  - Themes/Topics: {song_data['themes_topics']}
  - Summary: {song_data['summary']}

Provide exactly 7 song recommendations in a strict JSON array format. Each object in the array must contain only two fields: \"title\" and \"artist\".

Example Output:
[
  {{\"title\": \"Recommended Song 1\", \"artist\": \"Artist 1\"}},
  {{\"title\": \"Recommended Song 2\", \"artist\": \"Artist 2\"}},
  ...
]
"""

**prompt - stage 4**

gives data that the ai can parse through to use as information for there reccomendations then give the explected output of a list with a vairety of dictionary

In [ ]:
raw_recommendations_json = call_gemini_with_retry(recommendation_prompt)
print("Raw recommendations JSON received.")

try:
    recommendations = json.loads(raw_recommendations_json)
    print("Successfully parsed song recommendations JSON.")
    # Optionally, you can print the recommendations to verify
    # print(json.dumps(recommendations, indent=2))
except json.JSONDecodeError as e:
    print(f"JSON decoding error in recommendations: {e}")
    recommendations = [] # Initialize as empty list on error


Raw recommendations JSON received.
Successfully parsed song recommendations JSON.


**Recommendations - stage 4**

prints all the recommendations

In [ ]:
print(recommendations)

[{'title': 'Underneath the Tree', 'artist': 'Kelly Clarkson'}, {'title': 'Santa Tell Me', 'artist': 'Ariana Grande'}, {'title': 'Christmas (Baby Please Come Home)', 'artist': 'Darlene Love'}, {'title': 'My Only Wish (This Year)', 'artist': 'Britney Spears'}, {'title': 'This Christmas', 'artist': 'Donny Hathaway'}, {'title': 'Mistletoe', 'artist': 'Justin Bieber'}, {'title': 'Please Come Home for Christmas', 'artist': 'Charles Brown'}]


**prompt:**

gives information ai is allowed to parse and use to come to desion when making new lrycs and then has it write a new string

In [ ]:
new_verse_prompt = f"""
You are a skilled songwriter.

Write ONE new, original verse for the song "{song_data.get('title')}" by {song_data.get('artist')}.

Use the following analytical data as guidance:
Genres: {song_data.get('Genre')}
Sentiments: {song_data.get('sentiments')}
Themes/Topics: {song_data.get('themes_topics')}
Summary: {song_data.get('summary')}

Requirements:
- Match the tone, emotional mood, and thematic meaning of the original song.
- Maintain a consistent rhyme style and rhythm pattern if the song clearly uses one.
- Do NOT rewrite existing lyrics.
- Output ONLY the new verse text with no commentary, labels, or explanation.

New Verse:
"""

print("Generated prompt for new verse.")

Generated prompt for new verse.


**prints new verse**

saves string of new verse in string and prints it

In [ ]:
new_verse = call_gemini_with_retry(new_verse_prompt)
print("--- New Verse ---")
print(new_verse)
print("-------------------")

--- New Verse ---
I don't need the bells to be ringing
Or hear the local church choir singing
I won't ask for ribbons or bows
Just the magic your presence shows
-------------------


**prompt b:**

gives information ai is allowed to parse and use to come to desion when writing review and requires that the ai cite its sources in the lrycs to reduce the chances of it hallucinating

In [ ]:
review_prompt = f"""
You are a professional music critic.

Write a 250-word professional music critic review of the song "{song_data.get('title')}" by {song_data.get('artist')}.

Use the following analytical data to support your claims:
Genres: {song_data.get('Genre')}
Sentiments: {song_data.get('sentiments')}
Themes/Topics: {song_data.get('themes_topics')}
Summary: {song_data.get('summary')}

Requirements:
- Maintain a formal, evaluative tone used in professional music journalism.
- Incorporate the song’s sentiments, themes, and genre characteristics as evidence.
- Do NOT summarize the plot only; provide critique, interpretation, and stylistic analysis.
- Write in paragraph form with no bullet points.
- Output only the 250-word review, no preface or commentary.
"""

print("Generated prompt for music review.")

Generated prompt for music review.


**prints music review**

In [ ]:
music_review = call_gemini_with_retry(review_prompt)
print("--- Music Critic Review ---")
print(music_review)
print("---------------------------")

--- Music Critic Review ---
Mariah Carey’s "All I Want for Christmas Is You" stands as the unchallenged gold standard of modern holiday composition, succeeding precisely because of its clever genre synthesis. It employs the joyous, bell-laden orchestration characteristic of classic 'Christmas' standards while anchoring the performance in definitive late-20th-century 'Pop' and 'R&B' vocal acrobatics. The track is built upon a foundation of ecstatic anticipation, managing to synthesize a dynamic, swinging structure that avoids the stale sentimentality of lesser seasonal efforts.

Lyrically, the song is a deceptively simple manifesto of 'Prioritizing Relationships.' The pervasive theme of 'Materialism Rejection'—explicitly stating, "I don't care about the presents / Underneath the Christmas tree"—elevates the core message from simple seasonal cheer to profound 'Romantic Devotion.' Carey delivers this message with an infectious blend of 'Affectionate' warmth and powerful 'Longing' in the b

**prompt C:**

this uses the same information as the last two extra credit prompts and makes it so that the chosen song is in the playlist. it uses the same idea of prompt 4 but requires that there being a one sentence explanation

In [ ]:
playlist_prompt = f"""
You are an expert music curator and playlist generator.

Create a curated playlist of 10 songs that includes \"{song_data.get('title')}\" by {song_data.get('artist')}.

Use the following analytical data to guide your choices:
Genres: {song_data.get('Genre')}
Sentiments: {song_data.get('sentiments')}
Themes/Topics: {song_data.get('themes_topics')}
Summary: {song_data.get('summary')}

Requirements:
- The first song in the playlist MUST be \"{song_data.get('title')}\" by {song_data.get('artist')}. For this song, the justification should be \"This is the original song from which the playlist was generated.\".
- Recommend 9 additional real songs that connect through shared genre traits, emotional tone, or thematic content.
- Provide exactly one sentence of justification for each song showing how it relates to the original.
- Return ONLY a valid JSON array of 10 objects. Each object must contain:
  \"title\": \"Song Title\",
  \"artist\": \"Artist Name\",
  \"justification\": \"One-sentence explanation.\"

Do NOT include commentary, notes, or anything outside the JSON array.
"""

print("Generated prompt for playlist.")

Generated prompt for playlist.


**print result**
prints the list of new songs in dictionarys that has the title artis and justfication based off of the our selcted songs own merits

In [ ]:
raw_playlist_json = call_gemini_with_retry(playlist_prompt)
print("Raw playlist JSON received.")

try:
    playlist = json.loads(raw_playlist_json)
    print("Successfully parsed playlist JSON.")
    print(json.dumps(playlist, indent=2))
except json.JSONDecodeError as e:
    print(f"JSON decoding error in playlist: {e}")
    print(f"Raw JSON output: {raw_playlist_json}")
    playlist = [] # Initialize as empty list on error

Raw playlist JSON received.
Successfully parsed playlist JSON.
[
  {
    "title": "All I Want for Christmas Is You",
    "artist": "Mariah Carey",
    "justification": "This is the original song from which the playlist was generated."
  },
  {
    "title": "Santa Tell Me",
    "artist": "Ariana Grande",
    "justification": "This track shares the contemporary Pop/R&B genre blend and focuses on acute romantic longing specific to the holiday season."
  },
  {
    "title": "Underneath the Tree",
    "artist": "Kelly Clarkson",
    "justification": "This song mirrors the joyful, upbeat Pop energy and the strong theme of prioritizing the loved one's presence over material gifts."
  },
  {
    "title": "This Christmas",
    "artist": "Donny Hathaway",
    "justification": "It provides the soulful R&B foundation and emphasizes a message of deep, romantic devotion as the central joy of the holiday."
  },
  {
    "title": "Mistletoe",
    "artist": "Justin Bieber",
    "justification": "This se